In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msn

C:\Users\Acer\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# ML Models
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ML Pre processing
from surprise.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Hyperparameter tuning
from surprise.model_selection import GridSearchCV

In [3]:
df_ratings = pd.read_table("dataSet/user_ratedmovies.dat",sep="\t")

In [4]:
df_ratings.head(2)

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44


In [5]:
df_temp = df_ratings[['userID','movieID','rating']]
df_temp.to_csv('modified_rating_dataSet.csv',index=False)

In [6]:
df_movies = pd.read_table("dataSet/movies.dat",sep="\t")
pd.set_option('display.max_columns',None)

In [7]:
df_movies.rename(columns={'id': 'movieID'}, inplace=True)

In [8]:
df_movies.head(2)

,movieID,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,rtAllCriticsNumRotten,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,0,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,15,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...


In [9]:
#df_movies.isnull().sum()

In [10]:
df_temp2 = df_movies[['movieID','title']]
#df_temp2.to_csv('modified_movies_dataSet.csv',index=False)

In [11]:
df_temp2.head(2)

,movieID,title
0,1,Toy story
1,2,Jumanji


In [12]:
is_duplicate = df_temp2.duplicated()

In [13]:
print("Number of duplicate records:", sum(is_duplicate))

Number of duplicate records: 0


In [14]:
df_temp2.to_csv('modified_movies_dataSet.csv',index=False)

In [15]:
df = df_ratings[['userID','movieID','rating']]

In [16]:
df.head(4)

,userID,movieID,rating
0,75,3,1.0
1,75,32,4.5
2,75,110,4.0
3,75,160,2.0


In [17]:
df.isnull().sum()

userID     0
movieID    0
rating     0
dtype: int64

In [18]:
is_duplicate = df.duplicated()

In [19]:
print("Number of duplicate records:", sum(is_duplicate))

Number of duplicate records: 0


### Models Training

In [22]:
# surprise reader API to read the dataset
reader = Reader()

In [23]:
data = Dataset.load_from_df(df_ratings[['userID','movieID','rating']], reader)
#data.split(n_folds=5)

In [37]:
'''
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 
'''

"\nbenchmark = []\n# Iterate over all algorithms\nfor algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:\n    # Perform cross validation\n    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)\n    \n    # Get results & append algorithm name\n    tmp = pd.DataFrame.from_dict(results).mean(axis=0)\n    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))\n    benchmark.append(tmp)\n    \npd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') \n"

### Hyperparameter Tuning

In [38]:
param_grid = {'n_epochs': [10,12], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.5]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

0.8294402485393714
{'n_epochs': 12, 'lr_all': 0.005, 'reg_all': 0.4}


In [39]:
# best RMSE score
print(gs.best_score['rmse'])

0.8294402485393714


In [40]:
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

{'n_epochs': 12, 'lr_all': 0.005, 'reg_all': 0.4}


In [41]:
algo = gs.best_estimator['rmse']

In [42]:
# Data split 9/1
trainset, testset = train_test_split(data, test_size=0.01)

In [43]:
# Fitting our trainset
algo.fit(trainset)

In [44]:
predictions = algo.test(testset) 

In [45]:
accuracy.rmse(predictions)

RMSE: 0.8103


0.8103492524690616

In [46]:
accuracy.mae(predictions)

MAE:  0.6246


0.6246126966678506

In [48]:
test = pd.DataFrame(predictions)

In [47]:
test.head(2)

,uid,iid,r_ui,est,details
0,37934,376,3.0,3.103597,{'was_impossible': False}
1,68988,53956,4.0,3.545662,{'was_impossible': False}


In [49]:
import pickle
#open a file, where you ant to store the data
file = open('SVD_modelv3.pkl', 'wb')
# dump information to that file
pickle.dump(algo, file)
file.close()

In [50]:
 df[df['userID'] == 71525]

,userID,movieID,rating
854913,71525,50,5.0
854914,71525,110,5.0
854915,71525,111,4.0
854916,71525,165,5.0
854917,71525,168,3.0
...,...,...,...
855322,71525,49649,3.0
855323,71525,50794,3.0
855324,71525,51077,3.0
855325,71525,51575,4.0


In [51]:
algo.predict(71525,50)

Prediction(uid=71525, iid=50, r_ui=None, est=4.4169620042696645, details={'was_impossible': False})

In [53]:
# user is not in dataset
algo.predict(1, 5107)

Prediction(uid=1, iid=5107, r_ui=None, est=3.141312353080428, details={'was_impossible': False})

In [73]:
id_list = df['userID'].unique()

In [85]:
id_list[104]

3304

In [54]:
def get_recommendations(uid):
    prediction_list = []
    for mid in df_ratings['movieID'].unique():
        predic = algo.predict(uid,mid)
        prediction_list.append(predic)
    return prediction_list  

In [86]:
prediction = get_recommendations(3304)

In [87]:
prediction[:5]

[Prediction(uid=3304, iid=3, r_ui=None, est=2.449587750733541, details={'was_impossible': False}),
 Prediction(uid=3304, iid=32, r_ui=None, est=3.244791859728231, details={'was_impossible': False}),
 Prediction(uid=3304, iid=110, r_ui=None, est=3.1521091256738325, details={'was_impossible': False}),
 Prediction(uid=3304, iid=160, r_ui=None, est=2.0519005963602037, details={'was_impossible': False}),
 Prediction(uid=3304, iid=163, r_ui=None, est=2.7799481384502407, details={'was_impossible': False})]

In [88]:
df_pred=pd.DataFrame(prediction)

In [89]:
# Renaming our predictions to original names
df_pred=df_pred.rename(columns={'uid':'userID', 'iid':'movieID','est':'rating'}).sort_values('rating',ascending=False).head(10)

In [90]:
df_pred

,userID,movieID,r_ui,rating,details
953,3304,2351,None,3.486928,{'was_impossible': False}
94,3304,858,None,3.473173,{'was_impossible': False}
696,3304,6669,None,3.466626,{'was_impossible': False}
6390,3304,668,None,3.452529,{'was_impossible': False}
70,3304,318,None,3.450031,{'was_impossible': False}
258,3304,2360,None,3.446603,{'was_impossible': False}
550,3304,50,None,3.446038,{'was_impossible': False}
664,3304,3030,None,3.439486,{'was_impossible': False}
669,3304,3435,None,3.434246,{'was_impossible': False}
1068,3304,44555,None,3.430444,{'was_impossible': False}


In [91]:
df_final = df_pred.merge(df_movies ,on='movieID',how='left')

In [92]:
df_final.head(3)

,userID,movieID,r_ui,rating,details,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,rtAllCriticsNumRotten,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,3304,2351,None,3.486928,{'was_impossible': False},Le notti di Cabiria,50783,Las noches de Cabiria,http://ia.media-imdb.com/images/M/MV5BMjExNTA3...,1957,nights_of_cabiria,8.7,35,34,1,97,8.4,11,11,0,100,4.4,3465,94,http://content7.flixster.com/movie/29/88/29880...
1,3304,858,None,3.473173,{'was_impossible': False},The Godfather,68646,El padrino,http://ia.media-imdb.com/images/M/MV5BMTc3MDQz...,1972,godfather,9,66,66,0,100,8.4,11,11,0,100,4.4,172072,97,http://content9.flixster.com/movie/11/12/76/11...
2,3304,6669,None,3.466626,{'was_impossible': False},Ikiru,44741,Vivir,http://ia.media-imdb.com/images/M/MV5BMTcyMDU0...,1952,ikiru,8.6,30,30,0,100,8.3,9,9,0,100,4.5,5818,96,http://content9.flixster.com/movie/11/14/66/11...


In [93]:
movies_list = df_final['title'].values

In [63]:
movies_list

array(['The Godfather', 'Le notti di Cabiria', 'Ikiru',
       'The Shawshank Redemption', 'The Usual Suspects', 'Festen',
       'Y�jinb�', 'L�t den r�tte komma in', "L'atalante",
       'Pather Panchali'], dtype=object)

In [72]:
movies_list

array(['Le notti di Cabiria', 'The Godfather', 'Ikiru',
       'The Shawshank Redemption', 'Pather Panchali', 'Festen',
       'The Usual Suspects', 'Y�jinb�', 'Double Indemnity',
       'Das Leben der Anderen'], dtype=object)

In [84]:
movies_list

array(['Le notti di Cabiria', 'The Godfather', 'Ikiru', 'Pather Panchali',
       'The Shawshank Redemption', 'Festen', 'The Usual Suspects',
       'Y�jinb�', 'Dom za vesanje', 'Double Indemnity'], dtype=object)

In [94]:
movies_list

array(['Le notti di Cabiria', 'The Godfather', 'Ikiru', 'Pather Panchali',
       'The Shawshank Redemption', 'Festen', 'The Usual Suspects',
       'Y�jinb�', 'Double Indemnity', 'Das Leben der Anderen'],
      dtype=object)